First things first let's initialise our environment.

*Run the cell below to import our libraries and initialise our LUSID client*

In [ ]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')

![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Set up our Scope

In [ ]:
scope_id = str(uuid.uuid4())[:4]
scope_name = 'UK_High_Growth_Equities_Fund'
scope = '{}_{}'.format(scope_name, scope_id)
print (scope)

![Scope](img/multiplecurrencies-scope.gif)

## 2) Load Instruments

In [ ]:
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
instrument_universe.head(n=10)

In [ ]:
# Initialise our dictionary to hold our instrument definitions
instrument_definitions = {}

# Iterate over our instrument universe
for index, instrument in instrument_universe.iterrows():

    # Create our ISIN property using the default ISIN property definition
    isin_property = models.InstrumentProperty(
        key='Instrument/default/Isin',
        value=models.PropertyValue(
            label_value=instrument['isin']))
    
    # Create our ticker property using the default ticker property definition
    ticker_property = models.InstrumentProperty(
        key='Instrument/default/Ticker',
        value=models.PropertyValue(
            label_value=instrument['ticker']))

    # Create the definition for our instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers={'Figi': instrument['figi']},
        properties=[
            isin_property,
            ticker_property
        ]
    )

# Call LUSID to upsert our instrument defintions
response = client.upsert_instruments(requests=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Create our Base Fund Portfolio

In [ ]:
base_portfolio_name = '{}_base_fund'.format(scope_name)
print (base_portfolio_name)

In [ ]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_name,
    base_currency='GBP',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create our portfolio
portfolio_response = client.create_portfolio(
    scope=scope,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Set our Holdings

In [ ]:
#Import our holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

In [ ]:
holding_adjustments = []

holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=7)

for row, holding in holdings.iterrows():

    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set our holdings
response = client.set_holdings(
    scope=scope,
    code=base_portfolio_name,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustments)

prettyprint.set_holdings_response(response, scope, base_portfolio_name)

![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Add Corporate Actions

In [ ]:
corporate_action_source_id = '{}_corporate_action_source'.format(base_portfolio_name)

corporate_action_request = models.CreateCorporateActionSourceRequest(
    scope=scope,
    code=corporate_action_source_id)

response = client.create_corporate_action_source(
    request=corporate_action_request)
    
print (response)

![Corporate-Action-Source](img/multiplecurrencies-corporateactionsource.gif)

In [ ]:
corporate_actions = pd.read_csv('data/multiplecurrencies-corporateactions.csv')
corporate_actions.head()

In [ ]:
actions = []

for row, action in corporate_actions.iterrows():
    
    luid = client.get_instrument(
        identifier_type='Figi',
        identifier=action['input_instrument_figi']).lusid_instrument_id
    
    transition = models.CorporateActionTransition(
        input_transition=models.CorporateActionTransitionComponent(
            instrument_uid=luid,
            units_factor=1,
            cost_factor=0),
        output_transitions=[
            models.CorporateActionTransitionComponent(
            instrument_uid=action['output_instrument_internal'],
            units_factor=action['dividend_yield'],
            cost_factor=0),
        ])
    
    actions.append(
        models.CreateCorporateAction(
            corporate_action_code=action['code'],
            announcement_date=datetime.now(pytz.UTC)+timedelta(days=1),
            ex_date=datetime.now(pytz.UTC)+timedelta(days=1),
            record_date=datetime.now(pytz.UTC)+timedelta(days=1),
            payment_date=datetime.now(pytz.UTC)+timedelta(days=1),
            transitions=[transition])
    )

response = client.batch_upsert_corporate_actions(
    scope=scope,
    code=corporate_action_source_id,
    actions=actions)

print (response)

In [ ]:
response = client.upsert_portfolio_details(
    scope=scope,
    code=base_portfolio_name,
    details=models.CreatePortfolioDetails(
        corporate_action_source_id=models.ResourceId(
            scope=scope,
            code=corporate_action_source_id),
        base_currency='GBP'))

print (response)

![Corporate-Actions](img/multiplecurrencies-corporateactions.gif)

## 6) Value our Base Fund Portfolio

### a) Analytic store

In [ ]:
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

In [ ]:
times = prices['time'].unique()

for time in times:
    quote_date = datetime.strptime(time, '%d/%m/%Y')

    # Create analytics store request
    analytics_store_request = models.CreateAnalyticStoreRequest(
        scope=scope,
        date_property=quote_date)

    # Call LUSID to create our analytics store
    client.create_analytic_store(request=analytics_store_request)
    
    instrument_analytics = []
    for row, quote in prices.loc[prices['time'] == time].iterrows():
        
        luid = client.get_instrument(
            identifier_type='Figi',
            identifier=quote['instrument_figi']).lusid_instrument_id
        
        instrument_analytics.append(
            models.InstrumentAnalytic(
                instrument_uid=luid,
                value=quote['price']))
        
    response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)
    
    print (response.key)
    

![Analytic-Stores](img/multiplecurrencies-analyticstores.gif)

### b) Aggregation

In [ ]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=datetime.now(pytz.UTC),
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='sum'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Holding/default/SubHoldingKey'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print(response.data)

![Analytic-Stores](img/multiplecurrencies-aggregatedbase.gif)

## 7) Securitise our Base Fund Portfolio

### a) Securitiese with x units

In [ ]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='default', code='number'))

# Call LUSID to create our new property
response = client.create_property_definition(definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print our key
prettyprint.heading('Circulation Property Key', circulation_property_key)

![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

In [ ]:
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': base_portfolio_name},
    properties=[models.InstrumentProperty(
        key=circulation_property_key,
        value=models.PropertyValue(
            metric_value=models.MetricValue(
                value=40000.00
                )))],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_name))

response = client.upsert_instruments(requests={'look-through':instrument_definition}
    )

In [ ]:
print (response.values)

![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

### b) Calculalate price from aggregation and store in the aggregation store

In [ ]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=datetime.now(pytz.UTC),
        metrics=[
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Portfolio/default/Name'
        ])

response = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print (response.data)

total_value = response.data[0]['Sum(Holding/default/PV)']
total_cost = response.data[0]['Sum(Holding/default/Cost)']

In [ ]:
instrument_analytics = []

quote_date = datetime(2019,2,18,tzinfo=pytz.UTC)

base_fund_instrument = client.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_name,
    instrument_property_keys=[circulation_property_key])

luid = base_fund_instrument.lusid_instrument_id
total_circulation = base_fund_instrument.properties[0].value
price = total_value / total_circulation

print (price)

instrument_analytics.append(
    models.InstrumentAnalytic(
        instrument_uid=luid,
        value=price))

response = client.set_analytics(
        scope=scope,
        year=quote_date.year,
        month=quote_date.month,
        day=quote_date.day,
        data=instrument_analytics)

#print (response.key)

![Base-Fund-Price](img/multiplecurrencies-basefundprice.gif)

## 8) Create Foreign Currency Portfolios

In [ ]:
fund_currencies = ['AUD', 'GBP', 'USD', 'EUR', 'JPY']

for currency in fund_currencies:
    
    portfolio_name = '{}_{}'.format(base_portfolio_name, currency)
    
    # Create the request to add our portfolio
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_name,
        base_currency=currency,
        description='The portfolio to hold our base fund in currency {}'.format(currency),
        created=portfolio_creation_date)

    # Call LUSID to create our portfolio
    portfolio_response = client.create_portfolio(
        scope=scope,
        create_request=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(portfolio_response)

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfolios.gif)

## 9) Add Base Fund Units

In [ ]:
holding_adjustments = []

holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=7)

holding_adjustments.append(
    models.AdjustHoldingRequest(
        instrument_identifiers={
                'Instrument/default/ClientInternal': base_portfolio_name},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=total_circulation/len(fund_currencies),
                    cost=models.CurrencyAndAmount(
                        amount=total_cost/len(fund_currencies),
                        currency='USD'),
                    portfolio_cost=total_cost/len(fund_currencies),
                    price=total_cost/total_circulation)
            ]
    )
)

for currency in fund_currencies:
    # Call LUSID to set our holdings
    response = client.set_holdings(
        scope=scope,
        code='{}_{}'.format(base_portfolio_name, currency),
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    prettyprint.set_holdings_response(response, scope, '{}_{}'.format(base_portfolio_name, currency))

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfoliosholdbase.gif)

## 10) Add Hedging Transactions

In [ ]:
response = client.get_holdings(
    scope=scope,
    code='{}_{}'.format(base_portfolio_name, 'GBP'))
    
print (response.values[0])

In [ ]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=scope,
        code='default'),
        effective_at=quote_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='sum'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='sum'),
        ],
        group_by=[
            'Holding/default/SubHoldingKey'
        ])


response_foreigncurrency = client.get_aggregation_by_portfolio(
    scope=scope,
    code='{}_{}'.format(base_portfolio_name, 'EUR'),
    request=aggregation_request)


response_basefund = client.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_name,
    request=aggregation_request)

print ('Foreign currency portfolio valuation with 8000 units of base fund')
for instrument in response_foreigncurrency.data:
    for key, value in instrument.items():
        print (key, value)
    print ('\n')

print ('\n\n')
    
print ('Valuation of base fund')
for instrument in response_basefund.data:
    for key, value in instrument.items():
        print (key, value)
    print ('\n')

## You can use this cell below to remove the look through portfolio id then re-run the cell above.

In [ ]:

instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': base_portfolio_name},
    properties=[models.InstrumentProperty(
        key=circulation_property_key,
        value=models.PropertyValue(
            metric_value=models.MetricValue(
                value=40000.00
                )))])

response = client.upsert_instruments(requests={'look-through':instrument_definition}
    )

## 11) Value Currency Portfolios

## 12) Securitise Currency Portfolios

## 13) Create Share Class Portfolios

## 15) Add Currency Fund Holdings

## 16) Value Each Share Class

## 17) Add transactions to end portfolio

## 18) Value each portfolio (bubble up)